In [136]:
import requests
import csv
import json
import random
from time import sleep
from collections import Counter
import re

from tqdm.notebook import tqdm
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from geopy.geocoders import Nominatim

In [59]:
df = pd.read_csv("csv/kijiji_listings_processed.csv")
del df["Unnamed: 0"]
df

,cost,address,unitType,bedrooms,bathrooms,hydro,heat,water,wifi_info,parking,...,pet,size,furnished,appliances,ac,outdoor,smoking,amenities,url,has_den
0,933,"22-41 Munroe Pl., Regina, SK, S4S 6A7",Apartment,2,1.0,0,1,1,Cable / TV,1,...,1,860,0,"['Laundry (In Building)', 'Fridge / Freezer']",1,['Balcony'],1,"['24 Hour Security', 'Storage Space']",/v-apartments-condos/regina/2-bedroom-22-41-mu...,0
1,850,"2175 Rae Street, Regina, SK, S4T 2E8",Apartment,1,1.0,0,1,1,Not Included,0,...,1,380,0,"['Laundry (In Building)', 'Fridge / Freezer']",1,['Not Included'],0,['Not Included'],/v-apartments-condos/regina/beautiful-bachelor...,0
2,799,"1424 Victoria Ave, Regina, SK, S4P 0P3",Apartment,1,1.0,0,1,1,Not Included,0,...,1,520,0,['Fridge / Freezer'],0,['Not Included'],0,['Not Included'],/v-apartments-condos/regina/downtown-apartment...,0
3,814,"3864 Rae Street, Regina, SK, S4S 3A1",Apartment,1,1.0,0,1,1,Not Included,0,...,1,675,0,['Fridge / Freezer'],0,['Not Included'],0,['Not Included'],/v-apartments-condos/regina/parliament-place-a...,0
4,1030,"2240 Robinson Street, Regina, SK, S4T 2P9",Apartment,2,1.0,0,1,1,Not Included,0,...,1,870,0,"['Laundry (In Building)', 'Fridge / Freezer']",1,['Not Included'],0,['Not Included'],/v-apartments-condos/regina/pet-friendly-2-bed...,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
673,1100,"125 Froom Crescent, Regina, SK S4N 1S8, Canada",Condo,2,1.0,0,1,1,Not Included,1,...,0,1000,1,"['Laundry (In Building)', 'Dishwasher', 'Fridg...",1,['Not Included'],0,"['Bicycle Parking', 'Storage Space']",/v-apartments-condos/regina/furnished-2-bedroo...,0
674,1095,", S4r2l7, Regina, SK",Apartment,2,1.0,0,1,1,Not Included,2,...,0,1000,0,"['Laundry (In Unit)', 'Dishwasher', 'Fridge / ...",1,['Yard'],0,['Not Included'],/v-apartments-condos/regina/clean-quiet-and-af...,0
675,895,"309 Belfast St, North Portal, SK S0C 1W0, Canada",House,2,1.0,0,0,1,Not Included,3,...,1,1300,0,"['Laundry (In Unit)', 'Dishwasher', 'Fridge / ...",0,['Yard'],0,['Not Included'],/v-apartments-condos/regina/house-for-rent/156...,1
676,1750,"Trinity Way, Regina, SK S0G 4G0, Canada",Condo,2,2.0,0,0,0,Not Included,2,...,1,1400,0,"['Laundry (In Unit)', 'Dishwasher', 'Fridge / ...",1,['Balcony'],0,['Elevator in Building'],/v-apartments-condos/regina/luxury-harbour-lan...,0


In [60]:
one_hot = pd.get_dummies(pd.Series(df["unitType"]), prefix='has_unitType')
# del df["unitType"]
df = df.join(one_hot)
df

,cost,address,unitType,bedrooms,bathrooms,hydro,heat,water,wifi_info,parking,...,smoking,amenities,url,has_den,has_unitType_Apartment,has_unitType_Basement,has_unitType_Condo,has_unitType_Duplex/Triplex,has_unitType_House,has_unitType_Townhouse
0,933,"22-41 Munroe Pl., Regina, SK, S4S 6A7",Apartment,2,1.0,0,1,1,Cable / TV,1,...,1,"['24 Hour Security', 'Storage Space']",/v-apartments-condos/regina/2-bedroom-22-41-mu...,0,1,0,0,0,0,0
1,850,"2175 Rae Street, Regina, SK, S4T 2E8",Apartment,1,1.0,0,1,1,Not Included,0,...,0,['Not Included'],/v-apartments-condos/regina/beautiful-bachelor...,0,1,0,0,0,0,0
2,799,"1424 Victoria Ave, Regina, SK, S4P 0P3",Apartment,1,1.0,0,1,1,Not Included,0,...,0,['Not Included'],/v-apartments-condos/regina/downtown-apartment...,0,1,0,0,0,0,0
3,814,"3864 Rae Street, Regina, SK, S4S 3A1",Apartment,1,1.0,0,1,1,Not Included,0,...,0,['Not Included'],/v-apartments-condos/regina/parliament-place-a...,0,1,0,0,0,0,0
4,1030,"2240 Robinson Street, Regina, SK, S4T 2P9",Apartment,2,1.0,0,1,1,Not Included,0,...,0,['Not Included'],/v-apartments-condos/regina/pet-friendly-2-bed...,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
673,1100,"125 Froom Crescent, Regina, SK S4N 1S8, Canada",Condo,2,1.0,0,1,1,Not Included,1,...,0,"['Bicycle Parking', 'Storage Space']",/v-apartments-condos/regina/furnished-2-bedroo...,0,0,0,1,0,0,0
674,1095,", S4r2l7, Regina, SK",Apartment,2,1.0,0,1,1,Not Included,2,...,0,['Not Included'],/v-apartments-condos/regina/clean-quiet-and-af...,0,1,0,0,0,0,0
675,895,"309 Belfast St, North Portal, SK S0C 1W0, Canada",House,2,1.0,0,0,1,Not Included,3,...,0,['Not Included'],/v-apartments-condos/regina/house-for-rent/156...,1,0,0,0,0,1,0
676,1750,"Trinity Way, Regina, SK S0G 4G0, Canada",Condo,2,2.0,0,0,0,Not Included,2,...,0,['Elevator in Building'],/v-apartments-condos/regina/luxury-harbour-lan...,0,0,0,1,0,0,0


In [65]:
df["has_wifiInfo_Internet"] = df["wifi_info"].str.contains("Internet").astype(int)
df["has_wifiInfo_Cable"] = df["wifi_info"].str.contains("Cable / TV").astype(int)
df["agreement_is_annual"] = (df["agreement"] == "1 Year").astype(int)
# del df["wifi_info"], df["unitType"], df["url"], df["agreement"]

In [109]:
to_encode = ["appliances", "amenities", "outdoor"]

for col in to_encode:
    df[col].astype(str).str.strip('[]')
    one_hot = df[col].astype(str).str.strip('[]').str.get_dummies(', ')
    one_hot.columns = one_hot.columns.str.replace("'", "")
    one_hot = one_hot.add_prefix(f"has_{col}_")
    df = df.join(one_hot)

In [117]:
del df["wifi_info"], df["unitType"], df["url"], df["agreement"]
for col in to_encode:
    del df[col]
    del df[f"has_{col}_Not Included"]

In [118]:
df.to_csv("csv/kijiji_listings_prepared.csv")

In [157]:
lats = []
longs = []
geolocator = Nominatim(user_agent="my-machine-learning-application")
bad_addresses = []

pattern = re.compile("\w\d\w( )?\d\w\d")

for index, row in tqdm(df.iterrows(), total=len(df)):
#     if index > 5:
#         break
    new_address = pattern.sub("Canada", row["address"])
    location = geolocator.geocode(new_address)
    if location == None:
        lat, long = -1, -1
        bad_addresses.append(index)
        print(f"Bad address: {row['address']}")
    else: 
        lat = location.latitude
        long = location.longitude
    lats.append(lat)
    longs.append(long)

  0%|          | 0/678 [00:00<?, ?it/s]

Bad address: S4X, Canada
Bad address: #109  5301 Universal Crescent, S4W0L2, Regina, SK
Bad address: 144 1st Ave north, S3N 1J8, Yorkton, SK
Bad address: 700 Gladmer Pk., Regina, SK, S4P 2W9
Bad address: 3068 Greenbank Road, Regina, SK, S4V 3R7
Bad address: 113 1 St W, Carnduff, SK S0C 0S0, Canada
Bad address: Waterloo Rd, Yorkton, SK S3N, Canada
Bad address: 563 Sorlien Ave, Macoun, SK S0C 1P0, Canada
Bad address: Green Rock Rd, Regina, SK S4V, Canada
Bad address: Dakota Dr, Regina, SK S4Z, Canada
Bad address: Balmoral St, Yorkton, SK S3N, Canada
Bad address: 700 Gladmer Pk., Regina, SK, S4P 2W9
Bad address: 1431 Rae, S4p 3g1, Regina, SK
Bad address: 1829 Second A Street, S4A 0J8, Estevan, SK
Bad address: Blake Cres, Regina, SK S4X, Canada
Bad address: S4T 2G 3, Canada
Bad address: 141 6 St, Weyburn, SK S4H 1B1, Canada
Bad address: Regina Airport, 5201 Regina Ave #1, Regina, SK S4W 1B3, Canada
Bad address: 2315 116, Regina S4P 3Y8 SK, Canada
Bad address: Edgar St, Regina, SK S4N, Cana

In [169]:
df["latitude"] = lats
df["longitude"] = longs
df.to_csv("csv/kijiji_listing_prepared.csv")

In [193]:
bad_addresses2 = []

for index, row in tqdm(df.iterrows(), total=len(df)):
    if row["latitude"] != -1.0:
        continue
    req = requests.get("https://geocoder.ca", params={"locate": row["address"], "json": 1})
    if "error" in req.json():
        bad_addresses2.append(row["address"])
        continue
    df.loc[index, "latitude"] = req.json()["latt"]
    df.loc[index, "longitude"] = req.json()["longt"]
#     print(req.json()["latt"], req.json()["longt"])


  0%|          | 0/678 [00:00<?, ?it/s]

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [194]:
print(bad_addresses2)

['S4X, Canada', 'S4T 2G 3, Canada']


In [219]:
postal = pd.read_csv("csv/CanadianPostalCodes202105.csv")
df = pd.read_csv("csv/kijiji_listing_prepared.csv")

In [217]:
%%timeit
row = postal[postal["POSTAL_CODE"] == "S4T 2G3"].iloc[0]
row["LATITUDE"], row["LONGITUDE"]

39.3 ms ± 1.93 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [259]:
unfindable_addresses = []
undocumented_addresses = []
postal_patt = re.compile("([ABCEGHJ-NPRSTVXY]\d[ABCEGHJ-NPRSTV-Z])[ -]?(\d[ABCEGHJ-NPRSTV-Z]\d)", re.I)

for index, row in tqdm(df.iterrows(), total=len(df)):
    match = postal_patt.search(row.address)
    if not match:
        unfindable_addresses.append(row.address)
        continue
#     print(f"{match.group(1)} {match.group(2)}")
    nrow = postal[postal["POSTAL_CODE"] == f"{match.group(1).upper()} {match.group(2).upper()}"]
    assert len(nrow) < 2
    if len(nrow) <= 0:
        undocumented_addresses.append(row.address)
        continue
    nrow = nrow.iloc[0]
    df.loc[index, "latitude"] = nrow.LATITUDE
    df.loc[index, "longitude"] = nrow.LONGITUDE

  0%|          | 0/678 [00:00<?, ?it/s]

In [260]:
unfindable_addresses, undocumented_addresses

(['S4X, Canada',
  'Carnduff, SK, Canada',
  'Waterloo Rd, Yorkton, SK S3N, Canada',
  'Green Rock Rd, Regina, SK S4V, Canada',
  'Dakota Dr, Regina, SK S4Z, Canada',
  'Balmoral St, Yorkton, SK S3N, Canada',
  'Blake Cres, Regina, SK S4X, Canada',
  'S4T 2G 3, Canada',
  'Edgar St, Regina, SK S4N, Canada',
  'Dakota Dr, Regina, SK S4Z, Canada',
  'Dakota Dr, Regina, SK S4Z, Canada',
  'Murphy Cres.& Lothian Mews, Regina, SK, S4X 1F5',
  'Dakota Dr, Regina, SK S4Z, Canada',
  '1 St, Estevan, SK S4A, Canada',
  'Rosemont, Canada',
  'Curtiss Ave, Regina, SK S0G, Canada',
  '2101 Heseltine Rd, Regina, SK S4V, Canada',
  'Oxbow, Canada',
  'angus road, Canada'],
 ['1630 Badham Blvd, Regina S4P0J6 SK',
  '1431 Rae, S4p 3g1, Regina, SK',
  ', S0Z7G0, Pilot Butte, SK, Canada'])

In [263]:
def findCoords(postal_code):
    return postal[postal["POSTAL_CODE"] == postal_code].iloc[0]

In [264]:
findCoords("H1V 2X8")

POSTAL_CODE        H1V 2X8
CITY              MONTREAL
PROVINCE_ABBR           QC
TIME_ZONE                5
LATITUDE         45.556383
LONGITUDE       -73.537271
Name: 803773, dtype: object

In [279]:
ndf = df.drop((df[df.longitude > -100].index))
ndf = ndf.drop((df[df.longitude < -120].index))

In [277]:
ndf.longitude.nlargest(10)

547   -101.555317
106   -101.667030
488   -101.667030
559   -101.667030
112   -101.687175
531   -101.687175
26    -101.690370
632   -101.709676
127   -101.797263
59    -102.017606
Name: longitude, dtype: float64

In [280]:
ndf.to_csv("csv/kijiji_fixed.csv")